In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, pointbiserialr

# =============================================================================
# PHASE 4: STATISTICAL VALIDATION & HYPOTHESIS TESTING
# =============================================================================

# Initialize Phase 4 by loading the enriched feature set
df = pd.read_csv('loan_data_enriched.csv')

# --- 1. Categorical Variable Significance (Chi-Square) ---
# Objective: Prove that our engineered segments (DTI Tiers, Risk Segments, etc.)
# are statistically valid predictors rather than random noise.

featured_categorical = [
    'dti_category', 'dti_rate_warning', 'int_rate_category', 
    'int_rate_warning', 'tenure_tier', 'is_60_months', 'risk_segment'
]

print("--- CATEGORICAL FEATURE VALIDATION ---")
for col in featured_categorical:
    # analyze_categorical utilizes chi2_contingency to calculate p-values
    analyze_categorical(df_1, col, 'defaulted')    

# --- 2. Numerical Variable Discrimination (T-Tests & Point-Biserial) ---
# Objective: Quantify the mean differences between Defaulted and Solvent groups
# and measure the linear strength of association with the target.

continuous_features = [
    'dti', 'int_rate', 'annual_income', 'loan_amount', 
    'total_payment', 'installment', 'payment_to_income_ratio',   
    'loan_to_income_ratio', 'payment_completion_ratio'
]

def stat_test_numeric(df, col, target):
    """
    Performs a dual-layer statistical audit:
    1. Welch's T-Test: Checks if group means are significantly different.
    2. Point-Biserial Correlation: Measures the relationship between a 
       continuous feature and a binary target.
    """
    # Group Segmentation for Comparison
    charged_off = df[df[target] == 1][col]
    fully_paid  = df[df[target] == 0][col]
    
    # A. WELCH'S T-TEST (Handles unequal variances between classes)
    t_stat, p_val = ttest_ind(fully_paid, charged_off, equal_var=False)

    # Descriptive Momentum
    mean_0 = fully_paid.mean()
    mean_1 = charged_off.mean()
    
    print(f"\nAUDIT: {col.upper()}")
    print(f"  - Portfolio Mean (Non-Default): {mean_0:.4f}")
    print(f"  - Portfolio Mean (Defaulted)  : {mean_1:.4f}")
    print(f"  - T-Statistic                 : {t_stat:.2f}")
    print(f"  - P-value (Significance)      : {p_val:.2e}")

    # Interpretation Logic
    if p_val < 0.05:
        print(f"   ✅ STATISTICALLY SIGNIFICANT: {col} effectively discriminates between risk classes.")
    else:
        print(f"   ❌ INSIGNIFICANT: No reliable mean difference detected.")
        
    # B. POINT-BISERIAL CORRELATION ANALYSIS
    # Quantifying the linear association strength with the binary default target.
    correlation, p_value_corr = pointbiserialr(df[col], df[target])
    
    print(f"  ---- Association Strength (Point-Biserial) ----")
    print(f"  - Correlation Coefficient (r): {correlation:.4f}")
    print(f"  - Significance (p):            {p_value_corr:.2e}")



# Execute Numerical Audit
for col in continuous_features:
    stat_test_numeric(df, col, 'defaulted')

# --- 3. Multicollinearity & Relationship Mapping ---
# Objective: Identify redundant features (r > 0.8) to ensure model parsimony 
# and prevent coefficient instability during the modeling phase.

# Appending target to matrix to see direct linear correlations
continuous_features.append('defaulted')

print("\n--- FINAL CORRELATION HEATMAP (STATISTICAL AUDIT) ---")
correlation_matrix_featured = df[continuous_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_featured, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Feature Interaction & Multicollinearity Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()



# Automated Redundancy Filter
print("\n🔍 CRITICAL MULTICOLLINEARITY ANALYSIS (|r| > 0.5):")
for i in range(len(correlation_matrix_featured.columns)):
    for j in range(i+1, len(correlation_matrix_featured.columns)):
        corr_value = correlation_matrix_featured.iloc[i, j]
        if abs(corr_value) > 0.5:
            col1 = correlation_matrix_featured.columns[i]
            col2 = correlation_matrix_featured.columns[j]
            
            # Risk Mitigation Interpretation
            if abs(corr_value) > 0.8:
                interpretation = "VERY STRONG: High risk of redundancy. Prioritize one feature."
            elif abs(corr_value) > 0.6:
                interpretation = "STRONG: Potential feature overlap. Monitor Variance Inflation (VIF)."
            else:
                interpretation = "MODERATE: Natural financial relationship."
            
            print(f"   {col1:<25} ↔ {col2:<25}: r = {corr_value:>7.3f}")
            print(f"     → {interpretation}\n")